# Database System Concepts

*Authors: Abraham Silberschatz, Henry F. Korth, and S. Sudarshan*

**ISBN: 978-0-07-352332-3**

In [1]:
%defaultDatasource jdbc:h2:mem:db

## Appendix A: Detailed University Schema

![E-R Diagram for University](images/Figure_A_2.png)

In [2]:
CREATE TABLE IF NOT EXISTS classroom (
    building    VARCHAR(15),
    roomNumber  VARCHAR(7),
    capacity    NUMERIC(4,0),
    PRIMARY KEY (building, roomNumber)
);

CREATE TABLE IF NOT EXISTS department (
    deptName    VARCHAR(20),
    building    VARCHAR(15),
    budget      NUMERIC(12,2) CHECK (budget > 0),
    PRIMARY KEY (deptName)
);

CREATE TABLE IF NOT EXISTS course (
    courseId    VARCHAR(8),
    title       VARCHAR(50),
    deptName    VARCHAR(20),
    credits     NUMERIC(2,0) CHECK (credits > 0),
    PRIMARY KEY (courseId),
    FOREIGN KEY (deptName) REFERENCES department ON DELETE SET NULL
);

CREATE TABLE IF NOT EXISTS instructor (
    instructorId VARCHAR(5),
    name         VARCHAR(20) NOT NULL,
    deptName     VARCHAR(20),
    salary       NUMERIC(8,2) CHECK (salary > 29000),
    PRIMARY KEY (instructorId),
    FOREIGN KEY (deptName) REFERENCES department ON DELETE SET NULL
);

CREATE TABLE IF NOT EXISTS section (
    courseId    VARCHAR(8),
    sectionId   VARCHAR(8),
    semester    VARCHAR(6) CHECK (semester IN ('Fall', 'Winter', 'Spring', 'Summer')),
    year        NUMERIC(4,0) CHECK (year > 1701 AND year < 2100),
    building    VARCHAR(15),
    roomNumber  VARCHAR(7),
    timeSlotId  VARCHAR(4),
    PRIMARY KEY (courseId, sectionId, semester, year),
    FOREIGN KEY (courseId) REFERENCES course ON DELETE CASCADE,
    FOREIGN KEY (building, roomNumber) REFERENCES classroom ON DELETE SET NULL
);

CREATE TABLE IF NOT EXISTS teaches (
    instructorId VARCHAR(5),
    courseId     VARCHAR(8),
    sectionId    VARCHAR(8),
    semester     VARCHAR(6),
    year         NUMERIC(4,0),
    PRIMARY KEY (instructorId, courseId, sectionId, semester, year),
    FOREIGN KEY (courseId, sectionId, semester, year) REFERENCES section ON DELETE CASCADE,
    FOREIGN KEY (instructorId) REFERENCES instructor ON DELETE CASCADE
);

CREATE TABLE IF NOT EXISTS student (
    studentId   VARCHAR(5),
    name        VARCHAR(20) NOT NULL,
    deptName    VARCHAR(20),
    totalCredit NUMERIC(3,0) CHECK (totalCredit >= 0),
    PRIMARY KEY (studentId),
    FOREIGN KEY (deptName) REFERENCES department ON DELETE SET NULL
);

CREATE TABLE IF NOT EXISTS takes (
    studentId   VARCHAR(5),
    courseId    VARCHAR(8),
    sectionId   VARCHAR(8),
    semester    VARCHAR(6),
    year        NUMERIC(4,0),
    grade       VARCHAR(2),
    PRIMARY KEY (studentId, courseId, sectionId, semester, year),
    FOREIGN KEY (courseId, sectionId, semester, year) REFERENCES section ON DELETE CASCADE,
    FOREIGN KEY (studentID) REFERENCES student ON DELETE CASCADE
);

CREATE TABLE IF NOT EXISTS advisor (
    studentId    VARCHAR(5),
    instructorId VARCHAR(5),
    PRIMARY KEY (studentId),
    FOREIGN KEY (studentId) REFERENCES student ON DELETE CASCADE,
    FOREIGN KEY (instructorId) REFERENCES instructor ON DELETE SET NULL
);

CREATE TABLE IF NOT EXISTS prereq (
    courseId    VARCHAR(8),
    prereqId    VARCHAR(8),
    PRIMARY KEY (courseId, prereqId),
    FOREIGN KEY (courseId) REFERENCES course ON DELETE CASCADE,
    FOREIGN KEY (prereqId) REFERENCES course
);

CREATE TABLE timeSlot (
    timeSlotId  VARCHAR(4),
    day         VARCHAR(1),
    startHr     NUMERIC(2) CHECK (startHr >= 0 AND startHr < 24),
    startMin    NUMERIC(2) CHECK (startMin >= 0 AND startMin < 60),
    endHr       NUMERIC(2) CHECK (endHr >= 0 AND endHr < 24),
    endMin      NUMERIC(2) CHECK (endMin >= 0 AND endMin < 60),
    PRIMARY KEY (timeSlotId, day, startHr, startMin)
);

In [3]:
-- Clear Tables
DELETE FROM prereq;
DELETE FROM timeSlot;
DELETE FROM advisor;
DELETE FROM takes;
DELETE FROM student;
DELETE FROM teaches;
DELETE FROM section;
DELETE FROM instructor;
DELETE FROM course;
DELETE FROM department;
DELETE FROM classroom;
-- Classroom
INSERT INTO classroom VALUES ('Packard', '101', '500');
INSERT INTO classroom VALUES ('Painter', '514', '10');
INSERT INTO classroom VALUES ('Taylor', '3128', '70');
INSERT INTO classroom VALUES ('Watson', '100', '30');
INSERT INTO classroom VALUES ('Watson', '120', '50');
-- Department
INSERT INTO department VALUES ('Biology', 'Watson', '90000');
INSERT INTO department VALUES ('Comp. Sci.', 'Taylor', '100000');
INSERT INTO department VALUES ('Elec. Eng.', 'Taylor', '85000');
INSERT INTO department VALUES ('Finance', 'Painter', '120000');
INSERT INTO department VALUES ('History', 'Painter', '50000');
INSERT INTO department VALUES ('Music', 'Packard', '80000');
INSERT INTO department VALUES ('Physics', 'Watson', '70000');
-- Course
INSERT INTO course VALUES ('BIO-101', 'Intro. to Biology', 'Biology', '4');
INSERT INTO course VALUES ('BIO-301', 'Genetics', 'Biology', '4');
INSERT INTO course VALUES ('BIO-399', 'Computational Biology', 'Biology', '3');
INSERT INTO course VALUES ('CS-101', 'Intro. to Computer Science', 'Comp. Sci.', '4');
INSERT INTO course VALUES ('CS-190', 'Game Design', 'Comp. Sci.', '4');
INSERT INTO course VALUES ('CS-315', 'Robotics', 'Comp. Sci.', '3');
INSERT INTO course VALUES ('CS-319', 'Image Processing', 'Comp. Sci.', '3');
INSERT INTO course VALUES ('CS-347', 'Database System Concepts', 'Comp. Sci.', '3');
INSERT INTO course VALUES ('EE-181', 'Intro. to Digital Systems', 'Elec. Eng.', '3');
INSERT INTO course VALUES ('FIN-201', 'Investment Banking', 'Finance', '3');
INSERT INTO course VALUES ('HIS-351', 'World History', 'History', '3');
INSERT INTO course VALUES ('MU-199', 'Music Video Production', 'Music', '3');
INSERT INTO course VALUES ('PHY-101', 'Physical Principles', 'Physics', '4');
-- Instructor
INSERT INTO instructor VALUES ('10101', 'Srinivasan', 'Comp. Sci.', '65000');
INSERT INTO instructor VALUES ('12121', 'Wu', 'Finance', '90000');
INSERT INTO instructor VALUES ('15151', 'Mozart', 'Music', '40000');
INSERT INTO instructor VALUES ('22222', 'Einstein', 'Physics', '95000');
INSERT INTO instructor VALUES ('32343', 'El Said', 'History', '60000');
INSERT INTO instructor VALUES ('33456', 'Gold', 'Physics', '87000');
INSERT INTO instructor VALUES ('45565', 'Katz', 'Comp. Sci.', '75000');
INSERT INTO instructor VALUES ('58583', 'Califieri', 'History', '62000');
INSERT INTO instructor VALUES ('76543', 'Singh', 'Finance', '80000');
INSERT INTO instructor VALUES ('76766', 'Crick', 'Biology', '72000');
INSERT INTO instructor VALUES ('83821', 'Brandt', 'Comp. Sci.', '92000');
INSERT INTO instructor VALUES ('98345', 'Kim', 'Elec. Eng.', '80000');
-- Section
INSERT INTO section VALUES ('BIO-101', '1', 'Summer', '2009', 'Painter', '514', 'B');
INSERT INTO section VALUES ('BIO-301', '1', 'Summer', '2010', 'Painter', '514', 'A');
INSERT INTO section VALUES ('CS-101', '1', 'Fall', '2009', 'Packard', '101', 'H');
INSERT INTO section VALUES ('CS-101', '1', 'Spring', '2010', 'Packard', '101', 'F');
INSERT INTO section VALUES ('CS-190', '1', 'Spring', '2009', 'Taylor', '3128', 'E');
INSERT INTO section VALUES ('CS-190', '2', 'Spring', '2009', 'Taylor', '3128', 'A');
INSERT INTO section VALUES ('CS-315', '1', 'Spring', '2010', 'Watson', '120', 'D');
INSERT INTO section VALUES ('CS-319', '1', 'Spring', '2010', 'Watson', '100', 'B');
INSERT INTO section VALUES ('CS-319', '2', 'Spring', '2010', 'Taylor', '3128', 'C');
INSERT INTO section VALUES ('CS-347', '1', 'Fall', '2009', 'Taylor', '3128', 'A');
INSERT INTO section VALUES ('EE-181', '1', 'Spring', '2009', 'Taylor', '3128', 'C');
INSERT INTO section VALUES ('FIN-201', '1', 'Spring', '2010', 'Packard', '101', 'B');
INSERT INTO section VALUES ('HIS-351', '1', 'Spring', '2010', 'Painter', '514', 'C');
INSERT INTO section VALUES ('MU-199', '1', 'Spring', '2010', 'Packard', '101', 'D');
INSERT INTO section VALUES ('PHY-101', '1', 'Fall', '2009', 'Watson', '100', 'A');
-- Teaches
INSERT INTO teaches VALUES ('10101', 'CS-101', '1', 'Fall', '2009');
INSERT INTO teaches VALUES ('10101', 'CS-315', '1', 'Spring', '2010');
INSERT INTO teaches VALUES ('10101', 'CS-347', '1', 'Fall', '2009');
INSERT INTO teaches VALUES ('12121', 'FIN-201', '1', 'Spring', '2010');
INSERT INTO teaches VALUES ('15151', 'MU-199', '1', 'Spring', '2010');
INSERT INTO teaches VALUES ('22222', 'PHY-101', '1', 'Fall', '2009');
INSERT INTO teaches VALUES ('32343', 'HIS-351', '1', 'Spring', '2010');
INSERT INTO teaches VALUES ('45565', 'CS-101', '1', 'Spring', '2010');
INSERT INTO teaches VALUES ('45565', 'CS-319', '1', 'Spring', '2010');
INSERT INTO teaches VALUES ('76766', 'BIO-101', '1', 'Summer', '2009');
INSERT INTO teaches VALUES ('76766', 'BIO-301', '1', 'Summer', '2010');
INSERT INTO teaches VALUES ('83821', 'CS-190', '1', 'Spring', '2009');
INSERT INTO teaches VALUES ('83821', 'CS-190', '2', 'Spring', '2009');
INSERT INTO teaches VALUES ('83821', 'CS-319', '2', 'Spring', '2010');
INSERT INTO teaches VALUES ('98345', 'EE-181', '1', 'Spring', '2009');
-- Student
INSERT INTO student VALUES ('00128', 'Zhang', 'Comp. Sci.', '102');
INSERT INTO student VALUES ('12345', 'Shankar', 'Comp. Sci.', '32');
INSERT INTO student VALUES ('19991', 'Brandt', 'History', '80');
INSERT INTO student VALUES ('23121', 'Chavez', 'Finance', '110');
INSERT INTO student VALUES ('44553', 'Peltier', 'Physics', '56');
INSERT INTO student VALUES ('45678', 'Levy', 'Physics', '46');
INSERT INTO student VALUES ('54321', 'Williams', 'Comp. Sci.', '54');
INSERT INTO student VALUES ('55739', 'Sanchez', 'Music', '38');
INSERT INTO student VALUES ('70557', 'Snow', 'Physics', '0');
INSERT INTO student VALUES ('76543', 'Brown', 'Comp. Sci.', '58');
INSERT INTO student VALUES ('76653', 'Aoi', 'Elec. Eng.', '60');
INSERT INTO student VALUES ('98765', 'Bourikas', 'Elec. Eng.', '98');
INSERT INTO student VALUES ('98988', 'Tanaka', 'Biology', '120');
-- Takes
INSERT INTO takes VALUES ('00128', 'CS-101', '1', 'Fall', '2009', 'A');
INSERT INTO takes VALUES ('00128', 'CS-347', '1', 'Fall', '2009', 'A-');
INSERT INTO takes VALUES ('12345', 'CS-101', '1', 'Fall', '2009', 'C');
INSERT INTO takes VALUES ('12345', 'CS-190', '2', 'Spring', '2009', 'A');
INSERT INTO takes VALUES ('12345', 'CS-315', '1', 'Spring', '2010', 'A');
INSERT INTO takes VALUES ('12345', 'CS-347', '1', 'Fall', '2009', 'A');
INSERT INTO takes VALUES ('19991', 'HIS-351', '1', 'Spring', '2010', 'B');
INSERT INTO takes VALUES ('23121', 'FIN-201', '1', 'Spring', '2010', 'C+');
INSERT INTO takes VALUES ('44553', 'PHY-101', '1', 'Fall', '2009', 'B-');
INSERT INTO takes VALUES ('45678', 'CS-101', '1', 'Fall', '2009', 'F');
INSERT INTO takes VALUES ('45678', 'CS-101', '1', 'Spring', '2010', 'B+');
INSERT INTO takes VALUES ('45678', 'CS-319', '1', 'Spring', '2010', 'B');
INSERT INTO takes VALUES ('54321', 'CS-101', '1', 'Fall', '2009', 'A-');
INSERT INTO takes VALUES ('54321', 'CS-190', '2', 'Spring', '2009', 'B+');
INSERT INTO takes VALUES ('55739', 'MU-199', '1', 'Spring', '2010', 'A-');
INSERT INTO takes VALUES ('76543', 'CS-101', '1', 'Fall', '2009', 'A');
INSERT INTO takes VALUES ('76543', 'CS-319', '2', 'Spring', '2010', 'A');
INSERT INTO takes VALUES ('76653', 'EE-181', '1', 'Spring', '2009', 'C');
INSERT INTO takes VALUES ('98765', 'CS-101', '1', 'Fall', '2009', 'C-');
INSERT INTO takes VALUES ('98765', 'CS-315', '1', 'Spring', '2010', 'B');
INSERT INTO takes VALUES ('98988', 'BIO-101', '1', 'Summer', '2009', 'A');
INSERT INTO takes VALUES ('98988', 'BIO-301', '1', 'Summer', '2010', null);
-- Advisor
INSERT INTO advisor VALUES ('00128', '45565');
INSERT INTO advisor VALUES ('12345', '10101');
INSERT INTO advisor VALUES ('23121', '76543');
INSERT INTO advisor VALUES ('44553', '22222');
INSERT INTO advisor VALUES ('45678', '22222');
INSERT INTO advisor VALUES ('76543', '45565');
INSERT INTO advisor VALUES ('76653', '98345');
INSERT INTO advisor VALUES ('98765', '98345');
INSERT INTO advisor VALUES ('98988', '76766');
-- Time Slot
INSERT INTO timeSlot VALUES ('A', 'M', '8', '0', '8', '50');
INSERT INTO timeSlot VALUES ('A', 'W', '8', '0', '8', '50');
INSERT INTO timeSlot VALUES ('A', 'F', '8', '0', '8', '50');
INSERT INTO timeSlot VALUES ('B', 'M', '9', '0', '9', '50');
INSERT INTO timeSlot VALUES ('B', 'W', '9', '0', '9', '50');
INSERT INTO timeSlot VALUES ('B', 'F', '9', '0', '9', '50');
INSERT INTO timeSlot VALUES ('C', 'M', '11', '0', '11', '50');
INSERT INTO timeSlot VALUES ('C', 'W', '11', '0', '11', '50');
INSERT INTO timeSlot VALUES ('C', 'F', '11', '0', '11', '50');
INSERT INTO timeSlot VALUES ('D', 'M', '13', '0', '13', '50');
INSERT INTO timeSlot VALUES ('D', 'W', '13', '0', '13', '50');
INSERT INTO timeSlot VALUES ('D', 'F', '13', '0', '13', '50');
INSERT INTO timeSlot VALUES ('E', 'T', '10', '30', '11', '45 ');
INSERT INTO timeSlot VALUES ('E', 'R', '10', '30', '11', '45 ');
INSERT INTO timeSlot VALUES ('F', 'T', '14', '30', '15', '45 ');
INSERT INTO timeSlot VALUES ('F', 'R', '14', '30', '15', '45 ');
INSERT INTO timeSlot VALUES ('G', 'M', '16', '0', '16', '50');
INSERT INTO timeSlot VALUES ('G', 'W', '16', '0', '16', '50');
INSERT INTO timeSlot VALUES ('G', 'F', '16', '0', '16', '50');
INSERT INTO timeSlot VALUES ('H', 'W', '10', '0', '12', '30');
-- Prereq
INSERT INTO prereq VALUES ('BIO-301', 'BIO-101');
INSERT INTO prereq VALUES ('BIO-399', 'BIO-101');
INSERT INTO prereq VALUES ('CS-190', 'CS-101');
INSERT INTO prereq VALUES ('CS-315', 'CS-101');
INSERT INTO prereq VALUES ('CS-319', 'CS-101');
INSERT INTO prereq VALUES ('CS-347', 'CS-101');
INSERT INTO prereq VALUES ('EE-181', 'PHY-101');

## Chapter 1: Introduction

- A **database-management system** (DBMS) consists of a collection of interrelated data and a collection of programs to access that data.

### 1.3 View of Data

#### Data Abstraction

- **Physical Level**: *How is the data stored in the database?*
- **Logical Level**: *What is the data stored in the database?*
- **View Level**: *How should users of the database interact with the data?*

#### Instances and Schemas

- **Instance**: A collection of information stored in the database at a particular moment.
- **Schema**: The overall design of the database.
    - **Physical Schema**: Describes the database design at the physical level.
    - **Logical Schema**: Describes the database design at the logical level.
- **Physical Data Independence**: The ability to modify the physical schema without changing the logical schema or the application.
- **Logical Data Independence**: The ability to modify the logical schema without changing the application.

#### Data Models

- **Data Model**: A collection of conceptual tools for describing data, data relationships, data semantics, and consistency constraints.
    - Relational Model
    - Entity-Relationship Model
    - Object-Based Data Model
    - Semistructured Data Model

### 1.9 Database Architecture

![Database Architecture](images/Figure_1_5.png)

## Chapter 2: Introduction to the Relational Model

### 2.1 Structure of Relational Databases

- **Relation**: A set of tuples $\left( A_1, A_2, ..., A_n \right)$ in which each attribute $A_i$ is a member of the domain $D_i$.
    - *Relation = Table*
    - *Tuple = Row*
    - *Attribute = Column*
- **Relation Instance**: A specific set of tuples of a relation.
- **Domain**: A set of permitted values of an attribute.
- **Atomic**: A domain in which every element is semantically indivisible.
- **Null**: A special value that signifies that the value is unknown or does not exist.

### 2.2 Database Schema

- **Database Schema**: The logical design of a database.
- **Database Instance**: A snapshot of the data in a database at a given instant in time.
- **Relation Schema**: A list of attributes and their corresponding domains.

$$R \left( A_1, A_2, ..., A_n \right)$$

### 2.3 Keys

- A **superkey** of a relation is a set of one or more attributes whose values are guaranteed to identify tuples in the relation uniquely
- A **candidate key** is a minimal superkey, that is, a set of attributes that forms a superkey, but none of whose subsets is a superkey.
- A **primary key** is one of the candidate keys of a relation.
- A **foreign key** is a set of attributes in a referencing relation, such that for each tuple in the **referencing relation**, the values of the foreign key attributes are guaranteed to occur as the primary key value of a tuple in the **referenced relation**.

## Chapter 3: Introduction to SQL

### 3.1 Overview of the SQL Query Language

- **Data-Definition Language** (DDL) provides commands for defining relation schemas, deleting relations, and modifying relation schemas.
- **Data-Manipulation Language** (DML) includes a query language and commands to insert tuples into, delete tuples from, and modify tuples in the database.

### 3.2 SQL Data Definition

#### Basic Types

In [4]:
-- INT
-- BOOLEAN
-- TINYINT
-- SMALLINT
-- BIGINT
-- IDENTITY
-- DECIMAL
-- DOUBLE
-- REAL
-- TIME
-- DATE
-- TIMESTAMP
-- TIMESTAMP WITH TIME ZONE
-- BINARY
-- OTHER
-- VARCHAR
-- VARCHAR_IGNORECASE
-- CHAR
-- BLOB
-- CLOB
-- UUID
-- ARRAY
-- ENUM
-- GEOMETRY
-- INTERVAL

#### Basic Schema Definition

In [5]:
-- Visit https://en.wikipedia.org/wiki/Data_definition_language
-- for more information regarding DDL.
-- CREATE TABLE r
-- (
-- Attribute1 Domain1,
-- Attribute2 Domain2,
-- ...
-- AttributeN DomainN,
-- (IntegrityConstraint1),
-- ...
-- (IntegrityConstraintk),
-- );

### 3.3 Basic Structure of SQL Queries

#### Select Clause and From Clause

- The **`SELECT`** clause specifies the attributes to project for the output.
- The **`FROM`** clause specifies the relation from which to query.

In [6]:
-- List attribute deptName of all relation instructor.
SELECT deptName
FROM instructor;

In [7]:
-- List attribute deptName of all relation instructor.
-- Disallow Duplicates
SELECT DISTINCT deptName
FROM instructor;

In [8]:
-- List attribute deptName of all relation instructor.
-- Allow Duplicates
SELECT ALL deptName
FROM instructor;

#### Arithmetic Operations

In [9]:
-- List 1.
SELECT 1.0;

-- List Addition.
SELECT 1.0 + 1.0;

-- List Subtraction
SELECT 1.0 - 1.0;

-- List Multiplication.
SELECT 2.0 * 2.0;

-- List Division;
SELECT 1.0 / 2.0;

**\* Attribute**

- The **`*`** symbol denotes all attributes.

In [10]:
SELECT * FROM instructor;

#### Where Clause

- The **`WHERE`** clause specifies conditions by which the query should be filtered.

In [11]:
-- Connectives: 'and', 'or', 'not'
-- Operators: '<', '<=', '>', '>=', '=', '<>'
SELECT name
FROM instructor
WHERE deptName = 'Comp. Sci.' AND salary > 70000;

#### Cartesian Product

- The **Cartesian product** outputs all pairs of rows from the two input relations (regardless of whether or not they have the same values on common attributes).

In [12]:
SELECT name, courseId
FROM instructor, teaches
WHERE instructor.instructorId = teaches.instructorId 
    AND instructor.deptName = 'Comp. Sci.';

#### Natural Join

- The **natural join** outputs pairs of rows from the two input relations that have the same value on all attributes that have the same name.

In [13]:
SELECT name, courseId
FROM instructor NATURAL JOIN teaches
WHERE deptName = 'Comp. Sci.';

### 3.4 Additional Basic Operations

#### Rename Operation

- The **`AS`** operation aliases attributes and relations for efficiency and disambiguity.

In [14]:
-- What are the names of all instructors whose salary 
-- is greater than at least one instructor in the Biology 
-- department?
SELECT DISTINCT T.name
FROM instructor AS T, instructor AS S
WHERE T.salary > S.salary AND S.deptName = 'Biology';

#### Order By Clause

- The **`ORDER BY`** clause specifies the ordering by which the tuples in the result of a query should be sorted.

In [15]:
-- Alphabetically, what are the names of all instructors in the 
-- Physics department?
SELECT name
FROM instructor
WHERE deptName = 'Physics'
ORDER BY name;

-- Ascending
SELECT name
FROM instructor
WHERE deptName = 'Physics'
ORDER BY name ASC;

-- Descending
SELECT name
FROM instructor
WHERE deptName = 'Physics'
ORDER BY name DESC;

#### Where Clause 'Between' Predicate

In [16]:
-- What are the names of instructors with salary amounts 
-- between $90,000 and $100,000?
SELECT name
FROM instructor
WHERE salary BETWEEN 90000 AND 100000;

#### Where Clause 'Tuple Equality' Predicate

In [17]:
-- What are the instructor names and the courses they taught 
-- for all instructors in the Biology department who have 
-- taught some course?
SELECT name, courseId
FROM instructor, teaches
WHERE (instructor.instructorId, deptName) = (teaches.instructorId, 'Biology');

### 3.5 Set Operations

#### Union Operation

In [18]:
-- What are all the courses taught either in Fall 2009 or 
-- in Spring 2010, or both?
-- Disallow Duplicates
(SELECT courseId FROM section WHERE semester = 'Fall' AND year = 2009)
UNION
(SELECT courseId FROM section WHERE semester = 'Spring' AND year = 2010);

In [19]:
-- What are all the courses taught either in Fall 2009 or 
-- in Spring 2010, or both?
-- Allow Duplicates
(SELECT courseId FROM section WHERE semester = 'Fall' AND year = 2009)
UNION ALL
(SELECT courseId FROM section WHERE semester = 'Spring' AND year = 2010);

#### Intersect Operation

In [20]:
-- What are all the courses taught either in Fall 2009 and 
-- in Spring 2010?
-- Disallow Duplicates
(SELECT courseId FROM section WHERE semester = 'Fall' AND year = 2009)
INTERSECT
(SELECT courseId FROM section WHERE semester = 'Spring' AND year = 2010);

CS-101

#### Except Operation

In [21]:
-- What are all the courses taught either in Fall 2009 but not 
-- in Spring 2010?
-- Disallow Duplicates
(SELECT courseId FROM section WHERE semester = 'Fall' AND year = 2009)
EXCEPT
(SELECT courseId FROM section WHERE semester = 'Spring' AND year = 2010);

### 3.6 Null Values

In [22]:
SELECT NULL;

-- Addition, All NULL
-- SELECT NULL + NULL;
-- SELECT 1.0 + NULL;
-- SELECT NULL + 1.0;

-- Subtraction, All NULL
-- SELECT NULL - NULL;
-- SELECT 1.0 - NULL;
-- SELECT NULL - 1.0;

-- Multiplication, All NULL
-- SELECT NULL * NULL;
-- SELECT 1.0 * NULL;
-- SELECT NULL * 1.0;

-- Division, All NULL
-- SELECT NULL / NULL;
-- SELECT 1.0 / NULL;
-- SELECT NULL / 1.0;

-- And
SELECT NULL AND TRUE;
SELECT NULL AND FALSE;

-- Or
SELECT NULL OR TRUE;
SELECT NULL OR FALSE;

-- Not
SELECT NOT NULL;

-- Is
SELECT NULL IS NULL;
SELECT NULL IS NOT NULL;

### 3.7 Aggregate Functions

#### Basic Aggregation

In [23]:
-- Average
SELECT AVG(salary)
FROM instructor;

-- Minimum
SELECT MIN(salary)
FROM instructor;

-- Maximum
SELECT MAX(salary)
FROM instructor;

-- Sum
SELECT SUM(salary)
FROM instructor;

-- Count
SELECT COUNT(*)
FROM instructor;

#### Distinct Aggregation

In [24]:
-- What is the total number of instructors who teach a course 
-- in Spring 2010 semester?
SELECT COUNT(DISTINCT instructorId)
FROM teaches
WHERE (semester, year) = ('Spring', 2010);

6

#### Aggregation with  Grouping

- The **`GROUP BY`** clause specifies attributes by which tuples with the same value on all specified attributes are placed in the same group.

In [25]:
-- What is the average salary in each department?
SELECT deptName, AVG(salary) AS avgSalary
FROM instructor
GROUP BY deptName;

#### Having Clause

- The **`HAVING`** clause specifies conditions by which the query should be filtered after groups have been formed.

In [26]:
-- What is the average salary in each department 
-- if the average salary is greater than $42,000?
SELECT deptName, AVG(salary) AS avgSalary
FROM instructor
GROUP BY deptName
HAVING AVG(salary) > 42000;

#### Aggregation with Null Values

- All aggregate functions except `COUNT(*)` ignore null values in their input collection.

### 3.8 Nested Subqueries

- A **subquery** is a select-from-where expression that is nested within another query.

#### Set Membership

- The **`IN`** connective tests set membership.
- The **`NOT IN`** connective tests the absense of set membership.

In [27]:
-- What are all the courses taught either in Fall 2009 and 
-- in Spring 2010?
SELECT DISTINCT courseId
FROM section
WHERE semester = 'Fall'
    AND year = 2009
    AND courseId IN (
        SELECT courseId
        FROM section
        WHERE semester = 'Spring'
            AND year = 2010
    );

CS-101

#### Set Comparison

- The **`SOME`** connective asserts a condition for any member of a set.
- The **`ALL`** connective asserts a condition for all members of a set.

In [28]:
-- What are the names of all instructors whose salary 
-- is greater than at least one instructor in the Biology 
-- department?
SELECT name
FROM instructor
WHERE salary > SOME (
    SELECT salary
    FROM instructor
    WHERE deptName = 'Biology'
);

-- What are the names of all instructors whose salary 
-- is greater than all instructors in the Biology 
-- department?
SELECT name
FROM instructor
WHERE salary > ALL (
    SELECT salary
    FROM instructor
    WHERE deptName = 'Biology'
);

#### Existence Tests

- The **`EXISTS`** connective asserts whether a set is empty.
- The **`NOT EXISTS`** connective asserts whether a set is non-empty.

In [29]:
-- What are all the courses taught either in Fall 2009 and 
-- in Spring 2010?
SELECT courseId
FROM section AS S
WHERE semester = 'Fall'
    AND year = 2009
    AND EXISTS (
        SELECT *
        FROM section AS T
        WHERE semester = 'Spring'
            AND year = 2010
            AND S.courseId = T.courseId
    );

CS-101

#### Uniqueness Tests

- The **`UNIQUE`** connective asserts whether a set contains no duplicates.
- The **`NOT UNIQUE`** connective asserts whether a set contains duplicates.

In [30]:
-- DOES NOT WORK WITH H2!
-- What are all the courses that were offered at most once 
-- in 2009?
-- SELECT T.courseId
-- FROM course AS T
-- WHERE UNIQUE (
--     SELECT R.courseId
--     FROM section AS R
--     WHERE T.courseId = R.courseId
--         AND R.year = 2009
-- );

#### With Clause

- The **`WITH`** clause defines a temporary relation whose definition is available only to the query in which the clause occurs.

In [31]:
-- DOES NOT WORK WITH H2!
-- What department has the maximum budget?
-- WITH maxBudget(value) AS (
--     SELECT MAX(budget) FROM department
-- )
-- SELECT budget
-- FROM department, maxBudget
-- WHERE department.budget = maxBudget.value;

#### Scalary Subqueries

- A **scalar subquery** is a subequery that returns only one tuple containing a single attribute.
- A scalar subquery can be used wherever an expression returning a value is allowed.

In [32]:
-- What are all the departments along with the number of
-- instructors in each department?
SELECT deptName, (
    SELECT COUNT(*)
    FROM instructor
    WHERE department.deptName = instructor.deptName
) AS numInstructors
FROM department;

### 3.9 Modification of the Database

#### Deletion

- The **`DELETE`** statement deletes all tuples in a relation for which a given predicate is true.

In [33]:
SET AUTOCOMMIT FALSE;

SELECT CONCAT('Setup: ', COUNT(*))
FROM instructor;

-- 
DELETE FROM instructor;
-- 
SELECT CONCAT('Test 1: ', COUNT(*)) 
FROM instructor;
-- 
ROLLBACK;

-- 
DELETE FROM instructor
WHERE deptName = 'Finance';
-- 
SELECT CONCAT('Test 2: ', COUNT(*))
FROM instructor
WHERE deptName = 'Finance';
-- 
ROLLBACK;

-- 
DELETE FROM instructor
WHERE salary BETWEEN 13000 AND 15000;
-- 
SELECT CONCAT('Test 3: ', COUNT(*)) 
FROM instructor
WHERE salary BETWEEN 13000 AND 15000;
-- 
ROLLBACK;

-- 
DELETE FROM instructor
WHERE deptName IN (
    SELECT deptName
    FROM department
    WHERE building = 'Watson'
);
-- 
SELECT CONCAT('Test 4: ', COUNT(*)) 
FROM instructor
WHERE deptName IN (
    SELECT deptName
    FROM department
    WHERE building = 'Watson'
);
-- 
ROLLBACK;

SELECT CONCAT('Teardown: ', COUNT(*))
FROM instructor;

SET AUTOCOMMIT TRUE;

#### Insertion

- The **`INSERT`** statement inserts tuples into a relation.

In [34]:
SET AUTOCOMMIT FALSE;

SELECT CONCAT('Setup: ', COUNT(*))
FROM course;

-- 
INSERT INTO course
VALUES ('CS-437', 'Database Systems', 'Comp. Sci.', 4);
-- 
SELECT CONCAT('Test 1: ', COUNT(*)) 
FROM course;
-- 
ROLLBACK;

-- 
INSERT INTO course (courseId, title, deptName, credits)
VALUES ('CS-437', 'Database Systems', 'Comp. Sci.', 4);
-- 
SELECT CONCAT('Test 2: ', COUNT(*)) 
FROM course;
-- 
ROLLBACK;

SELECT CONCAT('Teardown: ', COUNT(*))
FROM course;

SET AUTOCOMMIT TRUE;

#### Update

- The **`UPDATE`** statement updates tuples of a relation.

In [35]:
SET AUTOCOMMIT FALSE;

SELECT CONCAT('Setup: ', COUNT(*))
FROM instructor;

-- 
UPDATE instructor
SET salary = salary * 1.05;
-- 
ROLLBACK;

-- 
UPDATE instructor
SET salary = salary * 1.05
WHERE salary < 70000;
-- 
ROLLBACK;

SELECT CONCAT('Teardown: ', COUNT(*))
FROM instructor;

SET AUTOCOMMIT TRUE;

## Chapter 4: Intermediate SQL

### 4.1 Join Expressions

In [36]:
SELECT *
FROM student NATURAL JOIN takes;

#### Join Conditions

- The **`JOIN ... USING`** clause specifies the required attributes to match for the join.
- The **`JOIN ... ON <condition>`** clause specifies the required condition to satisfy for the join.

#### Inner Joins

- The **`INNER JOIN`** do not preserve nonmatched tuples.

In [37]:
-- DOES NOT WORK WITH H2!
-- SELECT * 
-- FROM student INNER JOIN takes USING (studentId);

SELECT student.name, takes.courseId
FROM student INNER JOIN takes
    ON student.studentId = takes.studentId;

#### Outer Joins

- The **`LEFT OUTER JOIN`** preserves tuples only in the relation left of the *left outer join* operation.
- The **`RIGHT OUTER JOIN`** preserves tuples only in the relation right of the *left outer join* operation.
- The **`FULL OUTER JOIN`** preserves tuples in both relations.

In [38]:
SELECT student.name, takes.courseId
FROM student LEFT OUTER JOIN takes
    ON student.studentId = takes.studentId;
    
SELECT student.name, takes.courseId
FROM student RIGHT OUTER JOIN takes
    ON student.studentId = takes.studentId;
    
-- DOES NOT WORK WITH H2!
-- SELECT student.name, takes.courseId
-- FROM student FULL OUTER JOIN takes
--     ON student.studentId = takes.studentId;

## Chapter 6: Formal Relational Query Languages

### 6.1 The Relational Algebra

#### Assignment Operation

- **Notation**: $tempVariable \gets expression$
- **Definition**: Assigns a relational-algebra expression into a temporary relation variable.

#### Select Operation

- **Notation**: $\sigma_{predicate}(r)$
    - *And*: $\wedge$
    - *Or*: $\vee$
    - *Not*: $\neg$
- **Definition**: Return rows of the input relation that satisfy the predicate.
$$\sigma_{p}(r) = \left\{ t \mid t \in r \wedge p(t) \right\}$$
- **Example**: *What are all the instructors who are in the Physics department?*
$$\sigma_{deptName = 'Physics'}(instructor)$$

#### Project Operation

- **Notation**: $\prod_{attribute_1, attribute_2, ..., attribute_k}(r)$
- **Definition**: Output specfied attributes from all rows of the input relation. Remove duplicate tuples from the output.
- **Example**: *What are the IDs, names, and salaries of all instructors?*
$$\prod_{instructorId, name, salary}(instructor)$$

#### Union Operation

- **Notation**: $r \cup s$
- **Definition**: Output the union of tuples from the two input relations.
$$r \cup s = \left\{ t \mid t \in r \vee t \in s \right\}$$
- **Example**: *What are all the courses that are taught in Fall 2009 or in Spring 2010, or in both?*
$$\prod_{courseId}\left( \sigma_{semester = 'Fall' \wedge year = 2009}(section) \right) \cup \prod_{courseId}\left( \sigma_{semester = 'Spring' \wedge year = 2010}(section) \right)$$

#### Set Difference Operation

- **Notation**: $r - s$
- **Definition**: Output the tuples that are contained in the first relation but not in the second relation.
$$r - s = \left\{ t \mid t \in r \wedge t \notin s \right\}$$
- **Example**: *What are all the courses that are taught in Fall 2009 but not in Spring 2010?*
$$\prod_{courseId}\left( \sigma_{semester = 'Fall' \wedge year = 2009}(section) \right) - \prod_{courseId}\left( \sigma_{semester = 'Spring' \wedge year = 2010}(section) \right)$$

#### Cartesian Product Operation

- **Notation**: $r \times s$
- **Definition**: Output all pairs of rows from the two input relations.
$$r \times s = \left\{ tq \mid t \in r \wedge q \in s \right\}$$
- **Example**: *What are all the courses that all the instructors who are in the Physics department teach?*
$$I \gets instructor$$
$$T \gets teaches$$
$$\prod_{name, courseId}\left( \sigma_{I.instructorId = T.instructorId}\left( \sigma_{deptName = 'Physics'}(I \times T) \right) \right)$$

#### Rename Operation

- **Notation**: $\rho_{relation(attribute_1, attribute_2, ..., attribute_k)}(r)$
- **Definition**: Output the relation with a new relation name and a set of new attribute names. 
- **Example**: *What is the largest salary in the university?*
$$\prod_{salary} - \prod_{instructor.salary}\left( \sigma_{instructor.salary < d.salary}\left( instructor \times \rho_d(instructor) \right) \right)$$

#### Set Intersection Operation

- **Notation**: $r \cap s$
- **Definition**: Output the tuples that are contained in the first relation and in the second relation.
$$r \cap s = r - (r - s)$$
- **Example**: *What are all the courses that are taught in Fall 2009 and in Spring 2010?*
$$\prod_{courseId}\left( \sigma_{semester = 'Fall' \wedge year = 2009}(section) \right) \cap \prod_{courseId}\left( \sigma_{semester = 'Spring' \wedge year = 2010}(section) \right)$$

#### Set Division Operation

- **Requirement**: Every attribute of schema $S$ is in schema $R$.
- **Notation**: $r \div s$
- **Definition**: Outputs the largest relation $t(R - S)$ such that $t \times s \subseteq r$.
$$r \div s = \prod_{R - S}(r) - \prod_{R - S}\left( \left( \prod_{R - S}(r) \times s \right) - r \right)$$
    - Relation $r \div s$ is a relation on schema $R - S$.
    - A tuple $t$ is in $r \div s$ if and only if both conditions hold:
        1. $t$ is in $\prod_{R - S}(r)$
        2. For every typle $t_s$ in $s$, there is a tuple $t_r$ in $r$ satisfying both of the following:
            - $t_r[S] = t_s[S]$
            - $t_r[R - S] = t$
- **Example**: *What are all the students who have taken all courses in the Biology department?*
$$\prod_{studentId, courseId}(takes) \div \prod_{courseId}\left( \sigma_{deptName = 'Biology'}(course) \right)$$

##### Advice

1. The set division operation is suited to queries that include the phrase "for all".
2. The set division operation is analogous to the following:
    1. Key $r$ by $R - S$ for groups of $t_r[R - S] \rightarrow g$.
    2. Output $t_r[R - S]$ for each $t_r[R - S] \rightarrow g$ where $s \subseteq g$.

#### Natural Join Operation

- **Notation**: $r \Join s$
    - *Theta Join*: $\Join_\theta$
    - *Left Outer Join*: $⟕$
    - *Right Outer Join*: $⟖$
    - *Full Outer Join*: $⟗$
- **Definition**: Output all pairs of rows from the two input relations that have the same value on each of the attributes in $R \cap S$.
$$r \Join s = \prod_{r.A, r.B, r.C, r.D, s.E}\left( \sigma_{r.B = s.B \wedge r.D = s.D}(r \times s) \right)$$
- **Example**: *What are all the names of all instructors in the Physics department and all the courses that they teach?*
$$\prod_{name, title}\left( \sigma_{deptName = 'Physics'}(instructor \Join teaches \Join course) \right)$$

## Chapter 8: Relational Database Design

### 8.1 Features of Good Relational Designs

#### Bad Relational Database Design

##### Repetition of Information

- A condition where the values of one attribute are determined by the values of another attribute in the same relation, and both values are repeated throughout the relation.
- **Problem**: Increases the storage required for the relation.
- **Problem**: Makes updating the relation more difficult.

##### Inability to Represent Information

- A condition where a relationship exists among only a proper subset of the attributes in a relation.
- **Problem**: All the unrelated attributes must be filled with null values otherwise a tuple without the unrelated information cannot be inserted into the relation.

##### Loss of Information

- A condition which results from the decomposition of one relation into two relations and which cannot be combined to recreate the original relation.
- **Problem**: Certain queries cannot be answered using the reconstructed relation that could have been answered using the original relation.

#### Lossless-Join Decomposition

- A decomposition of relational schema $R$ into relational schemas $R_1$ and $R_2$ such that for every instance $r(R)$ corresponding with instances $r_1(R_1)$ and $r_2(R_2)$, $r = r_1 \Join r_2$ holds.
- A decomposition $\{ R_1, R_2 \}$ is a lossless-join decomposition if $R_1 \cap R_2 \rightarrow R_1$ or $R_1 \cap R_2 \rightarrow R_2$.

### 8.2 Atomic Domains and First Normal Form

- A domain is **atomic** if elements of the domain are considered to be indivisible units.
- A relation schema $R$ is in **first normal form** (1NF) if the domains of all attributes of $R$ are atomic.

### 8.3 Decomposition Using Functional Dependencies

#### Legal Instance and Superkey

- A **legal instance** is an instance of a relation that satisfies all such real-world constraints.
- A **superkey** $K$ of $r(R)$ is a subset of $R$ if, in any legal instance of $r(R)$, for all pairs $t_1$ and $t_2$ of tuples in the instance of $r$ if $t_1 \neq t_2$, then $t_1[K] \neq t_2[K]$.

#### Definition of Functional Dependency

- A **functional dependency** expresses constraints that uniquely identify the values of certain attributes.
$$\alpha \rightarrow \beta$$
- Given an instance of $r(R)$, the instance **satisfies** the functional dependency $\alpha \rightarrow \beta$ if for all pairs of tuples $t_1$ and $t_2$ in the instance such that $t_1[\alpha]$ and $t_2[\alpha]$, it is also the case that $t_1[\beta]$ and $t_2[\beta]$.
- The functional dependency $\alpha \rightarrow \beta$ **holds** on schema $r(R)$ if, in every legal instance of $r(R)$ it satisfies the functional dependency.
- A functional dependency is **trivial** if it is satisfied by all instances of a relation.
    - If $\beta \subseteq \alpha$, then $\alpha \rightarrow \beta$ is trivial.

#### Uses of Functional Dependencies

1. Test relations to see if they are legal under a given set of functional dependencies.
2. Specify constraints on the set of legal relations.

#### Closure of a Set of Functional Dependencies

- Given a relational schema $r(R)$, a functional dependency $f$ on $R$ is **logically implied** by a set of functional dependencies $F$ on $r$ if every instance of $r(R)$ that satisfies $F$ also satisfies $f$.
- The **closure** of $F$, denoted $F^+$, is the set of all functional dependencies logically implied by $F$.

#### Dependency Preservation

- A decomposition is **dependency preserving** if and only if $(F_1 \cup F_2 \cup ... \cup F_n)^+ = F^+$.

#### Boyce-Codd Normal Form

- A relation schema $R$ is in **Boyce-Codd normal form** (BCNF) with respect to a set $F$ of functional dependencies if for every dependency $\alpha \rightarrow \beta$ in $F^+$ such that $\alpha, \beta \subseteq R$, at least one of the following holds:
    - $\alpha \rightarrow \beta$ is trivial.
    - $\alpha$ is a superkey for $R$.

#### Third Normal Form

- A relation schema $R$ is in **third normal form** (3NF) with respect to a set $F$ of functional dependencies if for every dependency $\alpha \rightarrow \beta$ in $F^+$ such that $\alpha, \beta \subseteq R$, at least one of the following holds
    - $\alpha \rightarrow \beta$ is trivial.
    - $\alpha$ is a superkey for $R$.
    - Each attribute $B$ in $\beta - \alpha$ is contained in a candidate key for $R$.

#### Second Normal Form

- A **non-prime attribute** of a relation schema $R$ is an attribute that is not a part of any candidate key of $R$.
- A relation schema $R$ is in **second normal form** (2NF) with respect to a set $F$ of functional dependencies if:
    - $R$ is in 1NF.
    - $R$ depends on the whole of every candidate key.
- If every candidate key in $R$ has only one attribute, then $R$ is automatically in 2NF.

### 8.4 Functional-Dependency Theory

#### Armstrong's Axioms

- **Reflexivity**: If $\beta \subseteq \alpha$, then $\alpha \rightarrow \beta$.
- **Augmentation**: If $\alpha \rightarrow \beta$, then $\gamma\alpha \rightarrow \gamma\beta$.
- **Transitivity**: If $\alpha \rightarrow \beta$ and $\beta \rightarrow \gamma$, then $\alpha \rightarrow \gamma$.

#### Additional Inference Rules

- **Union**: If $\alpha \rightarrow \beta$ and $\alpha \rightarrow \gamma$, then $\alpha \rightarrow \beta\gamma$.
- **Decomposition**: If $\alpha \rightarrow \beta\gamma$, then $\alpha \rightarrow \beta$ and $\alpha \rightarrow \gamma$.
- **Pseudotransitivity**: If $\alpha \rightarrow \beta$ and $\gamma\beta \rightarrow \delta$, then $\gamma\alpha \rightarrow \delta$.

#### Computing $F^+$

![Computing $F^+$](images/Figure_8_7.png)

#### Closure of Attribute Sets

- Given a set of attributes $\alpha$, the **closure** of $\alpha$ under $F$, denoted $\alpha^+$, is the set of attributes that are functionally determined by $\alpha$ under $F$.

#### Uses of Attribute Set Closure

1. Test whether $\alpha$ is a superkey of $R$ by computing $\alpha^+$ and checking that $\alpha^+$ contains all attributes of $R$.
2. Test whether a functional dependency $\alpha \rightarrow \beta$ holds on $R$, compute $\alpha^+$ and check that $\beta \subseteq \alpha^+$.
3. Computing $F^+$:
    1. For each $\gamma \subseteq R$,
        1. Find the closure $\gamma^+$.
        2. For each $S \subseteq \gamma^+$, output the functional dependency $\gamma \rightarrow S$.

#### Computing $\alpha^+$

![Computing $\alpha^+$](images/Figure_8_8.png)

#### Canonical Cover

- A **canonical cover** of $F$ is a minimal set of functional dependencies equivalent to $F$, having no redundant dependencies.

#### Extraneous Attributes

- Given a set $F$ of functional dependencies and a functional dependency $\alpha \rightarrow \beta$ in $F$,
- Attribute $A$ is **extraneous** in $\alpha$ if $A \in \alpha$ and $F$ logically implies $\left( F - \{ \alpha \rightarrow \beta \} \right) \cup \left\{ (\alpha - A) \rightarrow \beta \right\}$.
    - Compute $(\{ \alpha \} - A)^+$ using the functional dependencies in $F$.
    - $A$ is extraneous in $\alpha$ if and only if $(\{ \alpha \} - A)^+$ contains $\beta$.
- Attribute $B$ is **extraneous** in $\beta$ if $B \in \beta$ and $\left( F - \{ \alpha \rightarrow \beta \} \right) \cup \left\{ \alpha \rightarrow (\beta - B) \right\}$ logically implies $F$.
    - Compute $\alpha^+$ using the functional dependencies in $\left( F - \{ \alpha \rightarrow \beta \} \right) \cup \left\{ \alpha \rightarrow (\beta - B) \right\}$.
    - $B$ is extraneous in $\beta$ if and only if $\alpha^+$ contains $B$.

#### Computing $F^C$

![Computing $F^C$](images/Figure_8_9.png)

### 8.5 Algorithms for Decomposition

#### General Test for BCNF

1. Given a relation schema $R$ and a set of functional dependencies $F$,
2. Compute attribute closures for all subsets of attributes of $R$ with respect to $F$.
3. Examine the attribute closures and look for a dependency $\alpha \rightarrow \beta$ that violates BCNF.
4. If no such dependency exists, then conclude that $R$ is in BCNF.

#### Alternative Test for BCNF

1. If $F$ is a set of functional dependencies over **ONLY** the attributes of $R$,
2. Examine each functional dependency $\alpha \rightarrow \beta$ and check whether that violates BCNF.

#### BCNF Decomposition

![BCNF Decomposition](images/Figure_8_11.png)

#### General Test for 3NF

1. Given a relation schema $R$ and a set of functional dependencies $F$,
2. Compute attribute closures for all subsets of attributes of $R$ with respect to $F$.
3. Identify all candidate keys.
4. Examine the attribute closures and look for a dependency $\alpha \rightarrow \beta$ that violates 3NF.
5. If no such dependency exists, then conclude that $R$ is in 3NF.

#### Alternative Test for 3NF

1. If $F$ is a set of functional dependencies over **ONLY** the attributes of $R$,
2. Identify all candidate keys.
3. Examine each functional dependency $\alpha \rightarrow \beta$ and check whether that violates 3NF.

#### 3NF Decomposition

![3NF Decomposition](images/Figure_8_12.png)

## Chapter 10: Storage and File Structure

### 10.1 Overview of Physical Storage Media

#### Storage Device Hierarchy

1. Cache
2. Main Memory
3. Flash Memory
4. Magnetic-Disk Storage
5. Optical Storage
6. Tape Storage

#### Storage Types

- **Primary Storage**: Cache, Main Memory
- **Secondary Storage** (*Online Storage*): Flash Memory, Magnetic-Disk Storage
- **Tertiary Storage** (*Offline Storage*): Optical Storage, Tape Storage
- **Volatile Storage**: Losses Data without Power
- **Nonvolatile Storage**: Preserves Data without Power

### 10.2 Magnetic Disk and Flash Storage

#### Magnetic Disk

![Magnetic Disk](images/Figure_10_2.png)

#### Performance Measures of Disks

- **Access Time**: Time from when a read or write request is issued to when data transfer begins.
- **Seek Time**: Time for repositioning the arm.
- **Rotational Latency Time**: Time spent waiting for the sector to be accessed to appear under the head.
- **Average Latency Time**: One-half the time for a full rotation of the disk.
- **Data-Transfer Rate**: Rate at which data can be retrieved from or stored to the disk.
- **Mean Time to Failure** (*MTTF*): Average time expected of the system to run continuously without any failure.

#### Optimization of Disk-Block Access

- **Block**: A logical unit consisting of a fixed number of contiguous sectors, typically ~KBs.
- **Sequential Access**: Successive Requests for Successive Block.
    - $1 \times Seek Time + N \times Transfer Time$
- **Random Access**: Successive Requests for Random Blocks.
    - $N \times Seek Time + N \times Transfer Time$

##### Techniques

- Buffering
- Read-Ahead
- Disk-Arm-Scheduling Algorithms (Elevator Algorithm)
- File Organization & Fragmentation
- Nonvolatile Write Buffers (NVRAM)
- Log Disk (Journaling File Systems)

### 10.3 RAID

- **RAID**: Redundant Arrays of Independent Disks

#### Improvement of Reliability via Redundancy

- **Mirroring**: Duplicate Disks

#### Improvement in Performance via Parallelism

- **Data Striping**: Write Data Across Multiple Disks
- **Bit-Level Striping**: Bit $i$ to Disk $i$
- **Block-Level Striping** Block $i$ to Disk $(i \mod n) + 1$

#### RAID Levels

- **RAID Level 5**
    - *Small Write Performance*: $\frac{N}{N - 1} \times$
    - *Large Write Performance*: $(N - 1) \times$
    - *Small Read Performance*: $N \times$
    - *Large Read Performance*: $(N - 1) \times$
- **RAID Level 10**: RAID Level 1 + RAID Level 0
    - *Write Performance*: $\frac{N}{2} \times$
    - *Read Performance*: $N \times$

![RAID Levels](images/Figure_10_3.png)

### 10.5 File Organization

- **File**: Sequence of Records
- **Block**: $\text{File} \rightarrow \text{Fixed-Length Blocks}$

#### Fixed-Length Records

- Record $i$ to Bytes $n \times (i - 1)$
- **File Header**: Stores Address of First Deleted Record
- **Free List**: Deleted Record $i$ $\rightarrow$ Deleted Record $(i + 1)$

#### Variable-Length Records

- **Record** = &lt; Fixed Length Attributes, **Null Bitmap**, Variable Length Attributes &gt;
    - *Variable Length Attributes*: Fixed Size (Offset, Length)
- **Slotted-Page Structure**: Organizes Variable-Length Records via Header:
    1. Number of Record Entries
    2. End of Free Space in Block
    3. Location & Size of Each Record

### 10.6 Organization of Records in Files

- **Heap File Organization**: Any record can be placed anywhere in the file where there is space for the record. There is no ordering of records.
- **Sequential File Organization**: Records are stored in sequential order, according to the value of a "search key" of each record.
- **Hashing File Organization**: A hash function is computed on some attribute of each record. The result of the hash function specifies in which block of the file the record should be placed.
- **Multitable Clustering File Organization**: Records of several different relations are stored in the same file; further, related records of the different relations are stored on the same block, so that one I/O operation fetches related records from all the relations.

#### Sequential File Organization

- **Search Key**: Any set of attributes by which a sequential file is sorted.
- **Insertion**:
    1. Locate the position where the record is to be inserted.
        1. If free space, insert.
        2. If no free space, insert into an **overflow block**.
    2. Update the pointer chain.
- **Deletion**: Pointer Chains

### 10.7 Data-Dictionary Storage

- The **data dictionary** (**system catalog**) keeps track of **metadata**, that is data about data, such as relation names, attribute names and types, storage information, integrity constraints, and user information.

### 10.8 Database Buffer

- **Buffer**: A part of main memory available for storage of copies of disk blocks.
- **Buffer Manager**: Subsystem responsible for the allocation of buffer space.

#### Buffer Manager

- *Buffer Replacement Strategy*:
    - Least Recently Used (LRU) for Sorting
    - Most Recently Used (MRU) for Nested-Loop Joins
- *Pinned Blocks*: No Write-Back During Updates
- *Forced Output of Block*: Write-Back for Resiliency